In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/wifi-loc/my-magnum
# Model parameters
NUM_FACTORS = 150
NUM_EPOCHS = 250
SEED = 42

import sys
sys.path.append("../../")
import os
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
import numpy as np
pd.set_option("max_rows", None)
pd.set_option("max_columns", None)
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.cornac.cornac_utils import predict_ranking
from reco_utils.common.timer import Timer
from reco_utils.common.constants import SEED

print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.6.11 | packaged by conda-forge | (default, Nov 27 2020, 18:57:37) 
[GCC 9.3.0]
Cornac version: 1.12.0


In [17]:
data = pd.read_csv('3col_15M_1500_11000.csv')
# data.drop(columns=['Unnamed: 0'], inplace=True) 
data.rename(columns = {'total_orders_of_this_product' : 'rating'}, inplace = True)
# data['rating'] = 5
data.head()

,userID,itemID,rating
0,0,3,5
1,0,4,5
2,0,5,5
3,0,23,5
4,0,35,5


In [18]:
train_set = cornac.data.Dataset.from_uir(data.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 275845
Number of items: 1302


In [21]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.001,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [22]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/250 [00:00<?, ?it/s]

Optimization finished!
Took 1873.3578 seconds for training.


In [23]:
import pickle 
with open("./trained/bprmodel250k1500u11000blr1e-3.pkl", 'wb') as f:
    pickle.dump(bpr, f)

In [3]:
# import pickle 
# bpr = pickle.load(open("./trained/bprmodel300e.pkl", 'rb'))

In [28]:
users = pd.read_csv('test_users_final.csv')

with Timer() as t:
    all_predictions = predict_ranking(bpr, users, usercol='userID', itemcol='itemID')
print("Took {} seconds for prediction.".format(t))

Took 10.3078 seconds for prediction.


In [29]:
tempdf = pd.concat(
            [
                data[['userID', 'itemID']],
                pd.DataFrame(
                    data=np.ones(data.shape[0]), columns=["dummycol"], index=data.index
                ),
            ],
            axis=1,
        )
merged = pd.merge(tempdf, all_predictions, on=['userID', 'itemID'], how="outer")
del tempdf
# del data
# del all_predictions
merged = merged[merged["dummycol"].isnull()].drop("dummycol", axis=1)
merged.head()

,userID,itemID,prediction
12929565,54,3,2.893706
12929566,54,5,0.646097
12929567,54,35,2.657943
12929568,54,39,1.396566
12929569,54,51,0.891097


In [30]:
merged_u = merged.groupby(by='userID', sort=False)['prediction'].apply(lambda grp: grp.nlargest(10)).reset_index(name='prediction').set_index('level_1')
merged_i = merged_u.merge(merged[['itemID']], left_index=True, right_index=True)
del merged_u
merged_i.drop(columns=['prediction'], inplace=True) 

a = np.array(['rec1', 'rec2', 'rec3', 'rec4', 'rec5', 'rec6', 'rec7', 'rec8', 'rec9', 'rec10'])
merged_i = merged_i.assign(num=a[np.arange(len(merged_i)) % len(a)])

merged_i = merged_i.pivot(index='userID', columns='num', values='itemID').reset_index()
merged_f = merged_i[['userID', 'rec1', 'rec2', 'rec3', 'rec4', 'rec5', 'rec6', 'rec7', 'rec8', 'rec9', 'rec10']]
merged_f.head(15)

num,userID,rec1,rec2,rec3,rec4,rec5,rec6,rec7,rec8,rec9,rec10
0,54,2838,2419,2700,6487,4205,65,751,3,2136,1026
1,56,251,438,901,1401,349,280,257,4,1595,51
2,69,853,319,511,290,70,41,280,2026,213,455
3,107,471,992,623,173,2771,4189,1133,2136,6481,1432
4,147,434,68,931,2503,3,155,563,471,319,1998
5,192,280,236,257,39,51,931,70,853,1401,226
6,199,3,39,789,349,51,198,320,114,477,466
7,224,931,251,438,238,639,407,80,51,2204,538
8,229,1375,693,1253,1672,507,414,319,180,51,2698
9,272,2838,826,492,653,4400,931,198,40,39,1424


In [31]:
len(merged_i.index)

10000

In [32]:
merged_f.isnull().values.any()

False

In [33]:
merged_f.to_csv('/media/actrec/DATA/submits/androbaza_14.csv', index=False)